In [28]:
import os
import pandas as pd
import re

#### Get current working directory

In [2]:
cwd = os.getcwd()

cwd

'C:\\Users\\randy\\Desktop\\Personal\\github\\demo_pandas'

#### Read local csv into pandas

In [3]:
df = pd.read_csv(
    os.path.join(
        'file://',
        cwd,
        'bank.csv'
    ),
    sep=','
)

#### Analyse schema

In [4]:
df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
deposit      object
dtype: object

#### Count records

In [5]:
df.shape

(11162, 17)

#### Sample records

In [6]:
df.head(3)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes


#### Get distinct value of each column

In [7]:
col_distinct_val = {
    col: None for col in df.columns
}

for col in col_distinct_val.keys():

    print('Analysing', col)
    col_distinct_val[col] = [i for i in df[col].unique()]

Analysing age
Analysing job
Analysing marital
Analysing education
Analysing default
Analysing balance
Analysing housing
Analysing loan
Analysing contact
Analysing day
Analysing month
Analysing duration
Analysing campaign
Analysing pdays
Analysing previous
Analysing poutcome
Analysing deposit


#### Identify categorical and numerical columns

In [8]:
def is_number(value):
    
    try:
        float(value)
        return True
    except Exception:
        return False

numerical_cols = []
categorical_cols = []

for col, values in col_distinct_val.items():

    print('Analysing', col)
    test = {is_number(value) for value in values}

    if test == {True}:
        numerical_cols.append(col)
    else:
        categorical_cols.append(col)

Analysing age
Analysing job
Analysing marital
Analysing education
Analysing default
Analysing balance
Analysing housing
Analysing loan
Analysing contact
Analysing day
Analysing month
Analysing duration
Analysing campaign
Analysing pdays
Analysing previous
Analysing poutcome
Analysing deposit


In [9]:
print('Numerical:', numerical_cols, '\n')
print('Categorical:', categorical_cols)

Numerical: ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'] 

Categorical: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']


#### Convert numerical column type to numerical

In [10]:
for col in numerical_cols:

    df[col] = pd.to_numeric(
        df[col],
        downcast='float'
    )
    
df.dtypes

age          float32
job           object
marital       object
education     object
default       object
balance      float32
housing       object
loan          object
contact       object
day          float32
month         object
duration     float32
campaign     float32
pdays        float32
previous     float32
poutcome      object
deposit       object
dtype: object

#### Summary of numerical columns

In [11]:
df[numerical_cols].describe()

,age,balance,day,duration,campaign,pdays,previous
count,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000
mean,41.231949,1528.538574,15.658036,371.993805,2.508421,51.330406,0.832557
std,11.913369,3225.413330,8.420739,347.128387,2.722077,108.758278,2.292007
min,18.000000,-6847.000000,1.000000,2.000000,1.000000,-1.000000,0.000000
25%,32.000000,122.000000,8.000000,138.000000,1.000000,-1.000000,0.000000
50%,39.000000,550.000000,15.000000,255.000000,2.000000,-1.000000,0.000000
75%,49.000000,1708.000000,22.000000,496.000000,3.000000,20.750000,1.000000
max,95.000000,81204.000000,31.000000,3881.000000,63.000000,854.000000,58.000000


#### Summary of categorical columns

In [12]:
for col in categorical_cols:

    print(col, '>>', col_distinct_val[col], '\n')

job >> ['admin.', 'technician', 'services', 'management', 'retired', 'blue-collar', 'unemployed', 'entrepreneur', 'housemaid', 'unknown', 'self-employed', 'student'] 

marital >> ['married', 'single', 'divorced'] 

education >> ['secondary', 'tertiary', 'primary', 'unknown'] 

default >> ['no', 'yes'] 

housing >> ['yes', 'no'] 

loan >> ['no', 'yes'] 

contact >> ['unknown', 'cellular', 'telephone'] 

month >> ['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr', 'sep'] 

poutcome >> ['unknown', 'other', 'failure', 'success'] 

deposit >> ['yes', 'no'] 



#### Standardise value of existing column

In [13]:
df['poutcome'] = df['poutcome'].apply(
    lambda x: 'unknown' if x == 'other' else x
)

df['poutcome'].unique()

array(['unknown', 'failure', 'success'], dtype=object)

#### Balances summary by education and marital

In [14]:
df.groupby(
    ['education', 'marital']
)['balance'].agg(
    ['sum', 'mean', 'min', 'max']
).reset_index().sort_values(
    by=['education', 'marital'],
    ascending=[True, False]
)

,education,marital,sum,mean,min,max
2,primary,single,271800.0,1379.695435,-887.0,26965.0
1,primary,married,1723931.0,1568.635986,-1489.0,66653.0
0,primary,divorced,288816.0,1415.764648,-779.0,37127.0
5,secondary,single,1942320.0,1139.859131,-1139.0,56831.0
4,secondary,married,4500996.0,1442.626953,-1965.0,81204.0
3,secondary,divorced,656210.0,1006.457031,-934.0,12039.0
8,tertiary,single,2637784.0,1806.701416,-880.0,36252.0
7,tertiary,married,3433041.0,1862.746094,-6847.0,51439.0
6,tertiary,divorced,738586.0,1913.435181,-2282.0,52587.0
11,unknown,single,274720.0,1749.808960,-461.0,45248.0


#### Top 3 balance by marital

In [15]:
tmp = df.copy()
tmp['rank'] = tmp.groupby('marital')['balance'].rank(
    method='dense',
    ascending=False
)

tmp[
    tmp['rank'] < 4
].sort_values(
    by=['marital', 'rank'],
    ascending=[False, True]
)[
    ['marital', 'job', 'age', 'balance', 'rank']
].reset_index(drop=True)

,marital,job,age,balance,rank
0,single,admin.,43.0,56831.0,1.0
1,single,technician,39.0,45248.0,2.0
2,single,management,29.0,36252.0,3.0
3,married,retired,84.0,81204.0,1.0
4,married,retired,84.0,81204.0,1.0
5,married,blue-collar,52.0,66653.0,2.0
6,married,entrepreneur,56.0,51439.0,3.0
7,divorced,self-employed,61.0,52587.0,1.0
8,divorced,self-employed,61.0,52587.0,1.0
9,divorced,retired,75.0,37127.0,2.0


#### Transform job

In [38]:
a = 'a '
a.

Help on built-in function split:

split(...) method of builtins.str instance
    S.split(sep=None, maxsplit=-1) -> list of strings
    
    Return a list of the words in S, using sep as the
    delimiter string.  If maxsplit is given, at most maxsplit
    splits are done. If sep is not specified or is None, any
    whitespace string is a separator and empty strings are
    removed from the result.



In [45]:
job_df = df[['job']].drop_duplicates()

job_df = job_df.sort_values('job')
job_df['upper'] = job_df['job'].apply(lambda x: x.upper())
job_df['alpha'] = job_df['job'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
job_df['lower'] = job_df['alpha'].apply(lambda x: x.lower())
job_df['trimmed'] = job_df['lower'].apply(lambda x: x.strip())
job_df['length'] = job_df['trimmed'].apply(lambda x: len(x))
job_df['split'] = job_df['trimmed'].apply(lambda x: x.split(sep=' '))
job_df['word_count'] = job_df['split'].apply(lambda x: len(x))
job_df['partial'] = job_df['trimmed'].apply(lambda x: x[0:2])
job_df['concat'] = job_df['trimmed'].apply(lambda x: '~'.join([x, 'temp']))

job_df

,job,upper,alpha,lower,trimmed,length,split,word_count,partial,concat
0,admin.,ADMIN.,admin,admin,admin,5,[admin],1,ad,admin~temp
11,blue-collar,BLUE-COLLAR,blue collar,blue collar,blue collar,11,"[blue, collar]",2,bl,blue collar~temp
39,entrepreneur,ENTREPRENEUR,entrepreneur,entrepreneur,entrepreneur,12,[entrepreneur],1,en,entrepreneur~temp
47,housemaid,HOUSEMAID,housemaid,housemaid,housemaid,9,[housemaid],1,ho,housemaid~temp
5,management,MANAGEMENT,management,management,management,10,[management],1,ma,management~temp
7,retired,RETIRED,retired,retired,retired,7,[retired],1,re,retired~temp
77,self-employed,SELF-EMPLOYED,self employed,self employed,self employed,13,"[self, employed]",2,se,self employed~temp
3,services,SERVICES,services,services,services,8,[services],1,se,services~temp
162,student,STUDENT,student,student,student,7,[student],1,st,student~temp
2,technician,TECHNICIAN,technician,technician,technician,10,[technician],1,te,technician~temp


#### Create new column for negative balance check

In [16]:
df['negative_balance'] = df['balance'].apply(
    lambda x: 1 if x < 0 else 0
)

df.dtypes

age                 float32
job                  object
marital              object
education            object
default              object
balance             float32
housing              object
loan                 object
contact              object
day                 float32
month                object
duration            float32
campaign            float32
pdays               float32
previous            float32
poutcome             object
deposit              object
negative_balance      int64
dtype: object

#### Check if negative balance is a one-off or common occurrence

In [17]:
df.groupby('negative_balance')['negative_balance'].count()

negative_balance
0    10474
1      688
Name: negative_balance, dtype: int64

#### Rename created column

In [18]:
df = df.rename(
    columns={'negative_balance': 'neg_bal'}
)

df.dtypes

age          float32
job           object
marital       object
education     object
default       object
balance      float32
housing       object
loan          object
contact       object
day          float32
month         object
duration     float32
campaign     float32
pdays        float32
previous     float32
poutcome      object
deposit       object
neg_bal        int64
dtype: object

#### Drop created column

In [19]:
df = df.drop(columns='neg_bal')

df.dtypes

age          float32
job           object
marital       object
education     object
default       object
balance      float32
housing       object
loan          object
contact       object
day          float32
month         object
duration     float32
campaign     float32
pdays        float32
previous     float32
poutcome      object
deposit       object
dtype: object

#### Create new dataframe

In [20]:
month_dict = [
    {'month': 'jan', 'month_num': 1},
    {'month': 'feb', 'month_num': 2},
    {'month': 'mar', 'month_num': 3},
    {'month': 'apr', 'month_num': 4},
    {'month': 'may', 'month_num': 5},
    {'month': 'jun', 'month_num': 6},
    {'month': 'jul', 'month_num': 7},
    {'month': 'aug', 'month_num': 8},
    {'month': 'sep', 'month_num': 9},
    {'month': 'oct', 'month_num': 10},
    {'month': 'nov', 'month_num': 11},
    {'month': 'dec', 'month_num': 12}
]

month_df = pd.DataFrame(month_dict)

month_df

,month,month_num
0,jan,1
1,feb,2
2,mar,3
3,apr,4
4,may,5
5,jun,6
6,jul,7
7,aug,8
8,sep,9
9,oct,10


#### Join dataframes

In [21]:
df = df.merge(
    month_df,
    on='month',
    how='inner'
)

df.head(3)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,month_num
0,59.0,admin.,married,secondary,no,2343.0,yes,no,unknown,5.0,may,1042.0,1.0,-1.0,0.0,unknown,yes,5
1,56.0,admin.,married,secondary,no,45.0,no,no,unknown,5.0,may,1467.0,1.0,-1.0,0.0,unknown,yes,5
2,41.0,technician,married,secondary,no,1270.0,yes,no,unknown,5.0,may,1389.0,1.0,-1.0,0.0,unknown,yes,5


#### Union dataframes

In [22]:
df = pd.concat([df, df])

df.shape

(22324, 18)

#### Deduplicate dataframe

In [23]:
df = df.drop_duplicates()

df.shape

(11162, 18)